In [1]:
from pandas import Series, DataFrame
import pandas as pd
import calendar
%pylab inline

from patsy import dmatrices
import warnings
%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


# Setup

In [2]:
#read data in, appended 2014/2015, removed GO from col names
crime2014 = pd.read_csv("https://raw.githubusercontent.com/sxzhu/mis381_project/master/Austin_Crime_2014.csv", dtype=unicode, encoding='utf-8', parse_dates=[4,7])

crime2015 = pd.read_csv("https://raw.githubusercontent.com/sxzhu/mis381_project/master/Austin_Crime_2015.csv", dtype=unicode, encoding='utf-8', parse_dates=[4,7])
crime = crime2014.append(crime2015,ignore_index=True)

crime = crime.rename(index=str, columns={"Highest NIBRS/UCR Offense Description": "General Offense Description",\
                                 "GO Highest Offense Desc":"Detailed Offense Description",\
                                 "GO Location":"Location","GO Report Date":"Report Date",\
                                 "GO Location Zip":"Zipcode","GO District":"District","GO Census Tract":"Census Tract",\
                                 "GO X Coordinate":"X Coordinate","GO Y Coordinate":"Y Coordinate",\
                                      "GO Primary Key":"Primary Key"})

crime = crime.drop('Location_1', axis=1)
crime['Date'] = pd.DatetimeIndex(crime['Report Date'])
crime['Month'] = pd.DatetimeIndex(crime['Date']).month
crime['Month'] = crime['Month'].apply(lambda x: calendar.month_abbr[x])
crime['Year'] = pd.DatetimeIndex(crime['Report Date']).year

## Merge Crimes

In [3]:
def merge_crimes(s):
    """Map some crimes together."""
    
    # Create a map of misspellings to correct spelling
    misspellings = {'Burglary / \nBreaking & Entering':'Burglary',
                    'Auto Theft':'Theft: Auto Parts',
                    'Homicide: Murder & Nonnegligent Manslaughter':'Murder',
                    'Agg Assault':'Aggravated Assault'}
    
    if s in misspellings:
        return misspellings[s]
    else:
        return s
    
# Apply the function to the Series
crime['General Offense Description'] = crime['General Offense Description'].map(merge_crimes)

def merge_theft(s):
    """Map some crimes together."""
    
    if (s[:5]=='Theft'):
        return s[:5]
    else:
        return s
    
# Apply the function to the Series
crime['General Offense Description'] = crime['General Offense Description'].map(merge_theft)

# Check that it worked
# mask = (crime['General Offense Description'] == 'Agg Assault')
# crime['General Offense Description'][mask]  # should be empty

crime['General Offense Description'].value_counts()

Theft                 61908
Burglary              10597
Aggravated Assault     3803
Robbery                1809
Rape                   1042
Murder                   55
Name: General Offense Description, dtype: int64

In [4]:
#Add the 'Streetname' column to both dataframes, keeping only the street name information
# removes leading numbers, leading 'BLOCK', leading hyphenated or / numbers, trailing 'NB', 'SB', 'EB', 'WB', trailing 'SVRD', and leading 'N', 'S', 'E', 'W'

crime['Streetname'] = crime['Location'].str.rsplit('\n', -1).str[0].str.replace('^[/\s\d-]+/?\d*\s+', '').str.replace('^\d+[A-Z]\s+', '').str.replace('^BLOCK\s+', '').str.replace('\s[NSEW]B\s*$', '').str.replace('\sSVRD\s*$', '').str.replace('^\s*[NSEW]\s+', '').str.replace(' HALF ST', ' ST').str.rstrip()
crime['PrimaryStreet'] = crime['Streetname'].str.rsplit(' / ', -1).str[0]
crime['SecondaryStreet'] = crime['Streetname'].str.rsplit(' / ', -1).str[1]

In [5]:
crime['cleared'] = crime['Clearance Status']!='N'

# Final Dataframe

In [6]:
crime.rename(columns={'General Offense Description': 'General_Offense_Description', 'Report Date':'Report_Date'}, inplace=True)
crime.head()

,Clearance Date,Clearance Status,Council District,Census Tract,District,Detailed Offense Description,Location,Zipcode,Primary Key,Report_Date,X Coordinate,Y Coordinate,General_Offense_Description,Date,Month,Year,Streetname,PrimaryStreet,SecondaryStreet,cleared
0,2014-04-28,N,1,18.350000000000001,E,AGG ROBBERY/DEADLY WEAPON,12151 N IH 35 SVRD NB ...,78753,20141061920,2014-04-17,3135985,10117220,Robbery,2014-04-17,Apr,2014,IH 35,IH 35,NaN,False
1,2014-05-20,N,1,21.13,I,ROBBERY BY ASSAULT,3300 BLOCK ROCKHURST LN ...,78723,20141150937,2014-04-25,3137985,10087946,Robbery,2014-04-25,Apr,2014,ROCKHURST LN,ROCKHURST LN,NaN,False
2,2014-05-13,N,3,9.02,C,ROBBERY BY THREAT,E 7TH ST / CHICON ST ...,78702,20141310316,2014-05-11,3120890,10068910,Robbery,2014-05-11,May,2014,7TH ST / CHICON ST,7TH ST,CHICON ST,False
3,2015-03-24,C,1,21.04,I,AGG ROBBERY/DEADLY WEAPON,WHELESS LN / BERKMAN DR ...,78723,20141670098,2014-06-16,3130566,10089446,Robbery,2014-06-16,Jun,2014,WHELESS LN / BERKMAN DR,WHELESS LN,BERKMAN DR,True
4,2014-10-02,N,3,9.02,G,AGG ROBBERY/DEADLY WEAPON,WALLER ST / E 2ND ST ...,78702,20142070292,2014-07-26,3117732,10068195,Robbery,2014-07-26,Jul,2014,WALLER ST / E 2ND ST,WALLER ST,E 2ND ST,False


In [7]:
Y, X = dmatrices('cleared ~ 0 + Zipcode + Report_Date + General_Offense_Description', crime, return_type='dataframe')
y = Y['cleared[True]'].values

In [8]:
X[:5]

,Zipcode[78610],Zipcode[78613],Zipcode[78617],Zipcode[78652],Zipcode[78653],Zipcode[78660],Zipcode[78701],Zipcode[78702],Zipcode[78703],Zipcode[78704],...,Report_Date[T.Timestamp('2015-12-27 00:00:00')],Report_Date[T.Timestamp('2015-12-28 00:00:00')],Report_Date[T.Timestamp('2015-12-29 00:00:00')],Report_Date[T.Timestamp('2015-12-30 00:00:00')],Report_Date[T.Timestamp('2015-12-31 00:00:00')],General_Offense_Description[T.Burglary],General_Offense_Description[T.Murder],General_Offense_Description[T.Rape],General_Offense_Description[T.Robbery],General_Offense_Description[T.Theft]
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
#split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [10]:
from sklearn import tree
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [11]:
result = model.fit(X_train, y_train)

In [12]:
from sklearn import metrics

prediction_train = model.predict(X_train)
print metrics.accuracy_score(y_train, prediction_train)

0.808260869565


In [13]:
prediction = model.predict(X_test)
print metrics.accuracy_score(y_test, prediction)

0.806577056387


In [15]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Create the folds in the training data
kfold = StratifiedKFold(n_splits=3, shuffle=True)

# Iterate over max_depth
for max_depth in [1, 2, 3, 4, 5]:
    model3 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
    scores = cross_val_score(model3, X_train, y_train, cv=kfold)
    print 'max_depth={} scores={} avg_score={}'.format(max_depth, scores, scores.mean())

max_depth=1 scores=[ 0.79119565  0.79119565  0.79119565] avg_score=0.791195652174
max_depth=2 scores=[ 0.80032609  0.80152174  0.80288043] avg_score=0.801576086957
max_depth=3 scores=[ 0.80809783  0.80722826  0.80641304] avg_score=0.807246376812
max_depth=4 scores=[ 0.80722826  0.81070652  0.80358696] avg_score=0.807173913043
max_depth=5 scores=[ 0.80847826  0.80478261  0.80809783] avg_score=0.807119565217


In [18]:
print 1-sum(crime['cleared'])/float(len(crime))

0.788635847199


In [19]:
crime['General_Offense_Description'].value_counts()

Theft                 61908
Burglary              10597
Aggravated Assault     3803
Robbery                1809
Rape                   1042
Murder                   55
Name: General_Offense_Description, dtype: int64